colab 런타임 : T4

# 1.1 Indexing - Loader 실습

실습에 활용할 파일 업로드

In [1]:
from google.colab import files

TEST_DATA_PATH = './data'

f = files.upload(TEST_DATA_PATH)

ModuleNotFoundError: No module named 'google.colab'

업로드한 파일이 제대로 존재하는지 체크

In [2]:
import os

def list_file_info(
    dir_path: str, ext: str | None = None, depth: int = 0
) -> list:
    """
    지정한 directory 의 경로 내에 존재하는 파일정보를 가져오는 함수

    :param dir_path: directory 의 경로
    :param ext: 가져올 파일의 확장자, 미지정시 전체
    :param depth: 탐색 깊이, default = 0, 모든 depth 탐색은 -1로 지정
    """
    info_list = []
    for file_name in os.listdir(dir_path):
        # 존재하는 파일인지 check
        if os.path.isfile(os.path.join(dir_path, file_name)):
            if ext is None:
                info_list.append(
                    {
                        "path": os.path.join(dir_path, file_name),
                        "name": os.path.splitext(file_name)[0],
                        "ext": file_name.split(".")[-1],
                    }
                )
            else:
                if file_name.lower().endswith("." + ext):
                    info_list.append(
                        {
                            "path": os.path.join(dir_path, file_name),
                            "name": os.path.splitext(file_name)[0],
                            "ext": file_name.split(".")[-1],
                        }
                    )

        elif os.path.isdir(os.path.join(dir_path, file_name)):
            if depth == 0:
                continue
            else:
                # case 1: depth 가 -1 일 경우 모든 dir 를 순회
                # case 2: depth 가 1 이상일 경우 최대 depth 값 만큼의 깊이로 재귀호출됨
                info_list += list_file_info(
                    os.path.join(dir_path, file_name), ext, depth - 1
                )

    return list(filter(lambda i: not i["name"].startswith("~$"), info_list))

file_info_list = list_file_info(TEST_DATA_PATH, ext="pdf")

for idx, file_info in enumerate(file_info_list):
  print(f"{idx+1}. {file_info['path'].replace(TEST_DATA_PATH+'/', '')}")


1. A4_멀티모달AI생성모델.pdf
2. A1_반도체.pdf
3. A7_EmbodiedAI구현기술.pdf
4. A3_멀티모달기술.pdf
5. A5_연합학습.pdf
6. A2_컴퓨팅방식.pdf
7. A6_EmbodiedAI개념.pdf


In [3]:
!pip install langchain_community pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_core.documents import Document

load_file_txt_list: list[list[Document]] = []

for file_info in file_info_list:
  loader = PyMuPDFLoader(file_info["path"])
  load_file_txt_list.append(loader.load())

# 1번째 file 의 1번째 페이지 파싱 결과
print('전처리 전:\n'+load_file_txt_list[0][0].page_content[:200])

# 전처리 추출 결과를 확인해보니깐, 불필요한 문자가 포함되어 있는 것을 확인 (예: 페이지 번호 등)
# 이를 제거합니다.
# 이 코드는 제공한 데이터셋에 특화된 전처리 코드이므로 다른 데이터를 활용할 경우 비활성화 해주세요

filter_texts = ["THE AI REPORT 2024-1 | 2024. 7. 25.\n", "2024년 AI 이슈를 용어와 함께 쉽게 이해하기\n"]

for page_txt_list in load_file_txt_list:
  for page_txt in page_txt_list:
    text = page_txt.page_content
    for filter_text in filter_texts:
      if text.find(filter_text):
        text = text.replace(filter_text, '')

    text = "\n".join(text.split('\n')[2:])
    page_txt.page_content = text

# 제거 결과 확인
print()
print('전처리 후:\n'+load_file_txt_list[0][0].page_content[:200])

전처리 전:
THE AI REPORT 2024-1 | 2024. 7. 25.
38
2. 멀티모달 AI 생성 모델(주요 생성형 AI 모델)
2.1. 트랜스포머(Transformer): 멀티모달 AI의 언어 천재, 문맥을 파악하여 맥락 있는 결과를 생성하다
① 왜 나오게 되었는가?
트랜스포머는 인공지능이 언어를 더 잘 이해하고 생성할 수 있도록 하기 위해 등장했다. 이전의

전처리 후:
2. 멀티모달 AI 생성 모델(주요 생성형 AI 모델)
2.1. 트랜스포머(Transformer): 멀티모달 AI의 언어 천재, 문맥을 파악하여 맥락 있는 결과를 생성하다
① 왜 나오게 되었는가?
트랜스포머는 인공지능이 언어를 더 잘 이해하고 생성할 수 있도록 하기 위해 등장했다. 이전의 자연어 처리 
모델은 RNN(순환 신경망, Recurrent Neur


# 1.2 Indexing - Chunking 실습

In [5]:
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


CHUNK_SIZE = 512
CHUNK_OVERLAP = 125

# splitter 생성
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", " ", ""],
    length_function=lambda x: len(tokenizer(x)["input_ids"]),
)

split_text_list = []

for page_txt_list in load_file_txt_list:
    page_cnt = len(page_txt_list)
    split_texts = splitter.split_documents(page_txt_list)
    split_text_list.append(split_texts)
    print(f"페이지 수: {len(page_txt_list)} -> 청크 수: {len(split_texts)}")


페이지 수: 5 -> 청크 수: 10
페이지 수: 7 -> 청크 수: 13
페이지 수: 6 -> 청크 수: 11
페이지 수: 6 -> 청크 수: 11
페이지 수: 4 -> 청크 수: 7
페이지 수: 4 -> 청크 수: 7
페이지 수: 5 -> 청크 수: 10


In [25]:
for id in range(0, 5):
  print('\n==== 청킹 결과 ('+str(id)+'번) ====:\n'+split_text_list[0][id].page_content)


==== 청킹 결과 (0번) ====:
2. 멀티모달 AI 생성 모델(주요 생성형 AI 모델)
2.1. 트랜스포머(Transformer): 멀티모달 AI의 언어 천재, 문맥을 파악하여 맥락 있는 결과를 생성하다
① 왜 나오게 되었는가?
트랜스포머는 인공지능이 언어를 더 잘 이해하고 생성할 수 있도록 하기 위해 등장했다. 이전의 자연어 처리 
모델은 RNN(순환 신경망, Recurrent Neural Network)이라는 방식을 사용했는데, 이는 마치 책을 한 문장씩 
차례대로 읽어나가는 것과 비슷했다. 이 방식은 긴 문장을 이해하는 데 어려움이 있었고, 처리 속도도 느렸다. 
예를 들어, “나는 어제 친구와 함께 영화를 보았다. 그것은 정말 재미있었다.”라는 문장에서 “그것”이 무엇을 가
리키는지 파악하는 데 어려움이 있었다. 트랜스포머는 이러한 문제를 해결하고, 더 나아가 텍스트뿐만 아니라 
이미지, 음성 등 다양한 형태의 데이터를 함께 처리할 수 있는 멀티모달 AI의 기반을 발전 및 고도화시킨다.
② 쉽게 설명하면?
트랜스포머는 문장 전체를 한 번에 이해하는 인공지능 모델이다. 이는 마치 사람이 문장을 읽을 때 전체적인 
맥락을 파악하는 것과 비슷하다. 예를 들어, “배가 바다를 항해한다”와 “배가 고프다”라는 두 문장에서 ‘배’라는 
단어의 의미가 다르다는 것을 문맥을 통해 이해하는 것과 같다. 트랜스포머는 이러한 방식으로 단어들 사이의 
관계를 파악하고, 문장의 의미를 정확하게 이해한다. 또한, 여러 문장을 동시에 처리할 수 있어 매우 빠르게 
작동한다.
③ 예시와 함께 좀 더 자세히 알아볼까?
트랜스포머의 핵심은 ‘어텐션 메커니즘(Attention Mechanism)’이라는 기술이다. 이는 마치 사람이 문장을 
읽을 때 중요한 부분에 집중하는 것과 같다. 예를 들어, “나는 파란 하늘을 보며 행복을 느꼈다”라는 문장에서

==== 청킹 결과 (1번) ====:
관계를 파악하고, 문장의 의미를 정확하게 이해한다. 또한, 여러 문장을 동시에 처리할 수 

# 1.3 Indexing - Embedding 실습

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

In [27]:
sample_embed_vector = embeddings.embed_query(split_text_list[0][0].page_content)
print(len(sample_embed_vector))
print(sample_embed_vector)

1024
[0.03007318824529648, -3.664111500256695e-05, 0.010532170534133911, -0.014660526998341084, 0.03664563223719597, -0.022226205095648766, -0.055312998592853546, 0.06849735230207443, 0.03371273726224899, -0.002091482514515519, 0.024185681715607643, 0.000638130703009665, -0.01992986351251602, 0.011674349196255207, -0.03622124344110489, -0.011269647628068924, -0.07575508207082748, 0.0067330473102629185, -0.024584880098700523, -0.025378774851560593, 0.03971973434090614, 0.003252933965995908, -0.03932134807109833, -0.018640289083123207, -0.006245885975658894, 0.0158940851688385, -0.021302586421370506, -0.03157446160912514, -0.009396188892424107, -0.01748298481106758, -0.009663679637014866, 0.01044426392763853, -0.03344186767935753, -0.05943981185555458, -0.0054292092099785805, 0.02830890379846096, 0.025016700848937035, 0.03441617265343666, -0.05709310993552208, 0.056345414370298386, 0.0058083017356693745, 0.05331774801015854, 0.008165845647454262, -0.024034636095166206, -0.011997032910585

# 1.4 Indexing - Save VectorDB 실습


본격적인 시작에 앞서 vector database 패키지를 다운로드 해줍니다

In [28]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:

In [29]:
from langchain_chroma import Chroma

COLLECTION_NAME="test"

# 저장소 생성
store = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings,
)

# 재실행을 대비하여 저장소 초기화
store.reset_collection()

# 모든 document 들을 저장소에 추가하기
for split_text in split_text_list:
    before_count = store._collection.count()
    store.add_documents(split_text)
    after_count = store._collection.count()
    print(f"저장소에 document 추가: 총합({before_count} -> {after_count})")


저장소에 document 추가: 총합(0 -> 10)
저장소에 document 추가: 총합(10 -> 23)
저장소에 document 추가: 총합(23 -> 34)
저장소에 document 추가: 총합(34 -> 45)
저장소에 document 추가: 총합(45 -> 52)
저장소에 document 추가: 총합(52 -> 59)
저장소에 document 추가: 총합(59 -> 69)


# 2 Retrieve - 검색 실습


In [30]:
results = store.similarity_search("Embodied 에 대한 정보를 자료를 기반으로 설명해줘", k=4)

for doc in results:
  print(doc.page_content)
  print()
  print("="*100)
  print()

이처럼 Embodied AI는 실제 환경에서 경험을 쌓으며 더욱 똑똑해지고 유연해진다.
④ 왜 중요한가?
Embodied AI의 중요성은 AI가 현실 세계에 더 가까워진다는 점에 있다. 이는 마치 책으로만 공부하던 학생이 
실제 현장에서 경험을 쌓는 것과 같다. 이를 통해 AI는 더 실용적이고 유연한 능력을 갖출 수 있다.
예를 들어, 재난 현장에서 활동하는 구조 로봇을 생각해보자. 이 로봇은 실제 재난 현장의 불규칙한 지형, 예측 
불가능한 상황 등을 직접 경험하며 학습한다. 이런 경험을 통해 로봇은 더 효과적으로 인명을 구조하고 위험을 
감지할 수 있게 된다.
또한 Embodied AI는 인간과 AI의 상호작용을 더욱 자연스럽게 만든다. 예를 들어, 노인 돌봄 로봇은 실제 
노인들과 대화하고 교감하면서 더 섬세하고 인간적인 돌봄 서비스를 제공할 수 있게 된다.
⑤ 어디에 활용되는가?
Embodied AI는 우리 생활 곳곳에서 활용될 수 있다:
Ÿ 가정: 로봇 청소기뿐만 아니라 요리, 빨래, 설거지 등을 돕는 가사 도우미 로봇으로 활용된다.


예를 들어, 로봇 청소기를 생각해보자. 로봇 청소기는 센서로 주변 장애물을 감지하고, 내장된 AI로 청소 경
로를 계획하며, 바퀴와 브러시를 움직여 실제로 청소를 수행한다. 이처럼 Embodied AI는 실제 세상에서 직접 
경험하고 학습하며 작업을 수행한다.
③ 예시와 함께 좀 더 자세히 알아볼까?
Embodied AI의 예시는 우리 주변에서 쉽게 찾아볼 수 있다.
Ÿ 로봇 강아지: 소니의 ‘아이보’라는 로봇 강아지가 있다. 아이보는 카메라로 주인을 인식하고, 마이크로 음성 
명령을 듣는다. AI로 상황을 판단하여 꼬리를 흔들거나 짖는 등의 반응을 보인다. 실제 강아지처럼 주변 
환경과 상호작용하며 학습하고 성장한다.
Ÿ 자율주행차: 테슬라의 자율주행차는 여러 개의 카메라, 레이더, 초음파 센서 등으로 주변 상황을 파악한다. 
AI가 이 정보를 분석하여 차량의 속도와 방향을 조절한다. 주행 경험이 쌓일수록 더 안전하

In [31]:
tmp_retriever = store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 4,
        "fetch_k": 10,
        "lambda_mult": 0.5
    },
)


mmr_results = tmp_retriever.invoke("Embodied 에 대한 정보를 자료를 기반으로 설명해줘")

for doc in mmr_results:
  print(doc.page_content)
  print()
  print("="*100)
  print()

이처럼 Embodied AI는 실제 환경에서 경험을 쌓으며 더욱 똑똑해지고 유연해진다.
④ 왜 중요한가?
Embodied AI의 중요성은 AI가 현실 세계에 더 가까워진다는 점에 있다. 이는 마치 책으로만 공부하던 학생이 
실제 현장에서 경험을 쌓는 것과 같다. 이를 통해 AI는 더 실용적이고 유연한 능력을 갖출 수 있다.
예를 들어, 재난 현장에서 활동하는 구조 로봇을 생각해보자. 이 로봇은 실제 재난 현장의 불규칙한 지형, 예측 
불가능한 상황 등을 직접 경험하며 학습한다. 이런 경험을 통해 로봇은 더 효과적으로 인명을 구조하고 위험을 
감지할 수 있게 된다.
또한 Embodied AI는 인간과 AI의 상호작용을 더욱 자연스럽게 만든다. 예를 들어, 노인 돌봄 로봇은 실제 
노인들과 대화하고 교감하면서 더 섬세하고 인간적인 돌봄 서비스를 제공할 수 있게 된다.
⑤ 어디에 활용되는가?
Embodied AI는 우리 생활 곳곳에서 활용될 수 있다:
Ÿ 가정: 로봇 청소기뿐만 아니라 요리, 빨래, 설거지 등을 돕는 가사 도우미 로봇으로 활용된다.


기존의 AI는 주로 컴퓨터 안에서만 존재하며 데이터를 분석하고 결과를 내놓았다. 이는 마치 책만 보고 세상을 
배우는 것과 같다. 하지만 실제 세상은 책에 나오는 것보다 훨씬 복잡하고 예측하기 어렵다.
예를 들어, 자율주행차가 도로에서 마주치는 상황은 매번 다르다. 갑자기 뛰어드는 동물, 예기치 못한 공사 
현장, 날씨 변화 등 수많은 변수가 있다. 이런 상황에서 AI가 효과적으로 대응하려면 실제 환경과 상호작용하며 
학습하고 적응하는 능력이 필요하다. 이런 필요성에 의해 Embodied AI가 탄생했다.
② 쉽게 설명하면?
Embodied AI는 말 그대로 ‘몸을 가진 AI’이다. 이는 로봇, 드론, 자율주행차 등 물리적인 형태를 가진 AI 
시스템을 말한다. 이들은 마치 우리 인간처럼 감각 기관(센서)으로 주변을 인식하고, 뇌(AI 알고리듬)로 상황을 
판단하며, 근육(모터 등)을 움직여 행동한다.
25) 골드만삭

# 3.1 Generation - Template 를 활용하여 Prompt 생성 실습

In [32]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from langchain_core.runnables import RunnablePassthrough

In [33]:
# template 활용법 1 - 입력 변수
template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate.from_template(template)
print(prompt)
print()
print(prompt.format(country="대한민국"))

input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도는 어디인가요?'

대한민국의 수도는 어디인가요?


In [34]:
# template 활용법 2 - 부분 변수 채움
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

print(prompt)
print(prompt.format(country1="한국"))
print()

prompt_partial = prompt.partial(country2="일본")
print(prompt_partial)
print(prompt_partial.format(country1="한국"))

input_variables=['country1'] input_types={} partial_variables={'country2': '미국'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'
한국과 미국의 수도는 각각 어디인가요?

input_variables=['country1'] input_types={} partial_variables={'country2': '일본'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'
한국과 일본의 수도는 각각 어디인가요?


In [35]:
# template 활용법 3 - 부분 변수 채움 & 함수
def get_today():
    """오늘 날짜 출력 함수"""
    from datetime import datetime
    return datetime.now().strftime("%b-%d")

prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

print(prompt.format(n=3))

오늘의 날짜는 Mar-15 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.


In [36]:
# template 활용법 4 - Chat & MessagePlaceholder
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)

print(chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
))

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


# 3.2 Generation - 검색 결과와 프롬프트 결합 실습

In [37]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import format_document


# retriever 옵션 설정
retriever = store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 4,
        "fetch_k": 10,
        "lambda_mult": 0.5
    },
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    # "당신은 질의 응답 업무의 보조자 입니다."
    "Use the following pieces of retrieved context to answer the question."
    # "검색된 다음 컨텍스트 를 사용 하여 질문에 응답 하십시오."
    "If you don't know the answer, say that you don't know."
    # "답을 모르면 모른다고 하세요"
    "Use three sentences maximum and keep the answer concise."
    # "최대 세 문장을 사용하고 답을 간결하게 유지합니다."
    "\n\n"
    "{context}\n"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


# 검색 결과에 사용할 template
docs_prompt = PromptTemplate.from_template(
    "page_content:\n{page_content}\n"
)

def format_docs(inputs: dict) -> str:
    return "\n".join(format_document(doc, docs_prompt) for doc in inputs["context"])


stuff_format = (
    RunnablePassthrough.assign(**{"context": format_docs}).with_config(
        run_name="format_inputs"
    )
    | prompt
).with_config(run_name="stuff_format")

rag_prompt_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_format,
    ).assign(**{"prompt": stuff_format})

results = rag_prompt_chain.invoke({"input": "Embodied 에 대한 정보를 자료를 기반으로 설명해줘"})

print(results['prompt'].to_string())


System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.If you don't know the answer, say that you don't know.Use three sentences maximum and keep the answer concise.

page_content:
이처럼 Embodied AI는 실제 환경에서 경험을 쌓으며 더욱 똑똑해지고 유연해진다.
④ 왜 중요한가?
Embodied AI의 중요성은 AI가 현실 세계에 더 가까워진다는 점에 있다. 이는 마치 책으로만 공부하던 학생이 
실제 현장에서 경험을 쌓는 것과 같다. 이를 통해 AI는 더 실용적이고 유연한 능력을 갖출 수 있다.
예를 들어, 재난 현장에서 활동하는 구조 로봇을 생각해보자. 이 로봇은 실제 재난 현장의 불규칙한 지형, 예측 
불가능한 상황 등을 직접 경험하며 학습한다. 이런 경험을 통해 로봇은 더 효과적으로 인명을 구조하고 위험을 
감지할 수 있게 된다.
또한 Embodied AI는 인간과 AI의 상호작용을 더욱 자연스럽게 만든다. 예를 들어, 노인 돌봄 로봇은 실제 
노인들과 대화하고 교감하면서 더 섬세하고 인간적인 돌봄 서비스를 제공할 수 있게 된다.
⑤ 어디에 활용되는가?
Embodied AI는 우리 생활 곳곳에서 활용될 수 있다:
Ÿ 가정: 로봇 청소기뿐만 아니라 요리, 빨래, 설거지 등을 돕는 가사 도우미 로봇으로 활용된다.

page_content:
기존의 AI는 주로 컴퓨터 안에서만 존재하며 데이터를 분석하고 결과를 내놓았다. 이는 마치 책만 보고 세상을 
배우는 것과 같다. 하지만 실제 세상은 책에 나오는 것보다 훨씬 복잡하고 예측하기 어렵다.
예를 들어, 자율주행차가 도로에서 마주치는 상황은 매번 다르다. 갑자기 뛰어드는 동물, 예기치 못한 공사 
현장, 날씨 변화 등 수많은 변수가 있다.

# 3.3 Generation - LLM API 연결 실습

In [38]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.2 MB/s eta 0:00:00


In [40]:
import os

os.environ["OPENAI_API_KEY"] = "{키입력}"  # 생성한 api 키

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

LLM_MODEL_NAME = "gpt-3.5-turbo"
LLM_TEMPERATURE = 0

model = ChatOpenAI(
  temperature=LLM_TEMPERATURE,  # 창의성
  model_name=LLM_MODEL_NAME,  # 모델명
)

In [44]:
result = model.invoke("Embodied 에 대한 정보를 자료를 기반으로 설명해줘")
print(result)

content='Embodied는 물리적인 존재나 형태를 가진다는 의미로 사용되는 용어입니다. 이 용어는 주로 신체적인 경험과 감각을 강조하는 맥락에서 사용됩니다. 예를 들어, "embodied cognition"은 인간의 사고와 지식이 신체적 경험과 상호작용을 통해 형성된다는 이론을 가리킵니다.\n\nEmbodied cognition 이론은 인간의 사고 과정이 단순히 뇌의 활동에 의해 결정되는 것이 아니라, 신체적 경험과 감각, 환경과의 상호작용 등이 모두 영향을 미치는 것으로 보는 관점을 제시합니다. 이는 우리가 세상을 이해하고 상호작용하는 방식이 우리의 신체적 경험과 감각에 근거한다는 것을 강조하는 것입니다.\n\n또한, embodied는 물리적인 형태를 가진다는 의미로도 사용됩니다. 예를 들어, "embodied robot"은 실제로 물리적인 몸체를 가지고 활동하는 로봇을 가리킵니다. 이러한 로봇은 주변 환경과 상호작용하며 일상 생활에서 다양한 작업을 수행할 수 있습니다.\n\n종합하면, embodied는 신체적 경험과 감각을 강조하거나 물리적 형태를 가진 것을 의미하는 용어로 다양한 분야에서 사용되고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 460, 'prompt_tokens': 26, 'total_tokens': 486, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_rea

In [ ]:
chain = model | StrOutputParser()
result = chain.invoke("Embodied 에 대한 정보를 자료를 기반으로 설명해줘")
print(result)

Embodied는 물리적인 존재나 형태를 가진다는 의미로 사용되는 용어입니다. 이 용어는 주로 신체적인 경험과 감각을 강조하는 맥락에서 사용됩니다. 예를 들어, "embodied cognition"은 인간의 사고와 인식이 신체적 경험과 상호작용에 의해 형성된다는 이론을 가리킵니다.

Embodied cognition 이론은 인간의 사고 과정이 단순히 뇌의 활동에 의해만 형성되는 것이 아니라, 신체적 경험과 감각, 환경과의 상호작용 등이 모두 영향을 미치는 것으로 보고합니다. 이는 우리가 세상을 이해하고 상호작용하는 방식이 우리의 신체적 경험과 감각에 근거한다는 것을 의미합니다.

또한, embodied는 신체적인 형태나 존재를 강조하는 것으로도 사용됩니다. 예를 들어, "embodied robot"은 물리적인 로봇이나 기계를 가리키며, "embodied language"는 언어가 신체적인 동작이나 행동과 연관되어 있는 것을 의미할 수 있습니다.

종합하면, embodied는 신체적 경험과 감각, 물리적 형태와 존재를 강조하는 용어로 다양한 분야에서 사용되고 있습니다.


# 3.4 Generation - RAG 실습

In [45]:
rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=stuff_format | model | StrOutputParser(),
).assign(**{"prompt": stuff_format})

result = rag_chain.invoke({"input": "Embodied 에 대한 정보를 자료를 기반으로 설명해줘"})
print(result["answer"])

Embodied AI는 실제 환경에서 경험을 쌓으며 AI가 더욱 똑똑하고 유연해지는 기술을 말합니다. 이는 마치 책으로만 공부하는 것이 아니라 실제 경험을 통해 학습하는 것과 유사합니다. Embodied AI는 로봇이나 자율주행차와 같은 물리적 형태를 가진 AI 시스템을 포함하며, 주변을 감지하고 판단하여 행동할 수 있게 합니다.
